In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, Dropout, Conv1D, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import time

In [8]:
tf.config.run_functions_eagerly(True)

In [9]:
df = pd.read_csv('IMDB.csv')

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
# Data preprocessing
blanks = []
for i, rv, sn in df.itertuples():
    if type(rv) == str and rv.isspace():
        blanks.append(i)
        
if len(blanks) > 0:
    print(f'There are {len(blanks)} empty space strings in the dataset')
    df.drop(blanks, inplace=True)

In [11]:
df.nunique()

review       49582
sentiment        2
dtype: int64

In [12]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [13]:
max_seq_length = df['review'].str.len().max()

print("Max sequence length:", max_seq_length)

Max sequence length: 13704


In [14]:
words = set(word.lower() for sentence in df['review'] for word in sentence.split())
n_unique_words = len(words)

print("Unique words:", n_unique_words)

Unique words: 390931


In [15]:
X = df['review']
y = df['sentiment']

display(X, y)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

display(y)

array([1, 1, 1, ..., 0, 0, 0])

In [17]:
# Tokenization
max_len = 150
num_words = 1000

tok = Tokenizer(num_words=num_words)
tok.fit_on_texts(df['review'])

In [18]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=101)

X_train_mat = tok.texts_to_sequences(X_train)
X_test_mat = tok.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_mat, maxlen=max_len)
X_test_padded = pad_sequences(X_test_mat, maxlen=max_len)

In [19]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [15]:
# GRU
def create_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(GRU(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [20]:
units = 64
dropout_rate = 0.5 

In [21]:
gru_model = create_gru_model(units, dropout_rate)
gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
# Train the models and measure time
start_time = time.time()

In [19]:
# GRU
gru_history = gru_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])


C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
219/219 [==============================] - 365s 2s/step - loss: 0.4760 - accuracy: 0.7580 - val_loss: 0.3689 - val_accuracy: 0.8371
Epoch 2/10
219/219 [==============================] - 238s 1s/step - loss: 0.3381 - accuracy: 0.8548 - val_loss: 0.3444 - val_accuracy: 0.8514
Epoch 3/10
219/219 [==============================] - 235s 1s/step - loss: 0.3168 - accuracy: 0.8678 - val_loss: 0.3403 - val_accuracy: 0.8511
Epoch 4/10
219/219 [==============================] - 240s 1s/step - loss: 0.3006 - accuracy: 0.8751 - val_loss: 0.3313 - val_accuracy: 0.8630
Epoch 5/10
219/219 [==============================] - 238s 1s/step - loss: 0.2890 - accuracy: 0.8802 - val_loss: 0.3235 - val_accuracy: 0.8684
Epoch 6/10
219/219 [==============================] - 238s 1s/step - loss: 0.2724 - accuracy: 0.8896 - val_loss: 0.3182 - val_accuracy: 0.8690
Epoch 7/10
219/219 [==============================] - 237s 1s/step - loss: 0.2584 - accuracy: 0.8924 - val_loss: 0.3385 - val_accuracy: 0.8670

In [20]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm

# Save the trained model to an HDF5 file
gru_model.save('gru_model.h5')

C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# Predict using the trained model on test data
y_pred_prob = gru_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))

118/118 [==============================] - 94s 798ms/step
Correct Predictions: 12990
Wrong Predictions: 2010
Accuracy: 86.60%


In [47]:

loaded_model = load_model('gru_model.h5')

In [48]:
review = str(input('Movie Review: '))

Movie Review: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is 

In [49]:
import re
from nltk.corpus import stopwords  # Import NLTK's stopwords
import nltk
nltk.download('stopwords')  # Download the stopwords data if not already downloaded

#regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)


Cleaned:  One of the other reviewers has mentioned that after watching just  Oz episode youll be hooked They are right as this is exactly what happened with mebr br The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the wordbr br It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to manyAryans Muslims gangstas Latinos Christians Italians Irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awaybr br I would say the main appeal of the show is due to the fact that it goes where other shows wouldnt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer  # Import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords data if not already downloaded
nltk.download('stopwords')

# Define your text preprocessing function
def preprocess_text(text):
    regex = re.compile(r'[^a-zA-Z\s]')
    text = regex.sub('', text)
    words = text.split(' ')
    english_stops = set(stopwords.words('english'))
    filtered = [w.lower() for w in words if w.lower() not in english_stops]
    filtered_text = ' '.join(filtered)
    return filtered_text

# Example text
review = "One of the other reviewers has mentioned that after watching just  Oz episode youll be hooked They are right as this is exactly what happened with mebr br The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the wordbr br It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to manyAryans Muslims gangstas Latinos Christians Italians Irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awaybr br I would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare Forget pretty pictures painted for mainstream audiences forget charm forget romanceOZ doesnt mess around The first episode I ever saw struck me as so nasty it was surreal I couldnt say I was ready for it but as I watched more I developed a taste for Oz and got accustomed to the high levels of graphic violence Not just violence but injustice crooked guards wholl be sold out for a nickel inmates wholl kill on order and get away with it well mannered middle class inmates being turned into prison bitches due to their lack of street skills or prison experience Watching Oz you may become comfortable with what is uncomfortable viewingthats if you can get in touch with your darker side"

# Preprocess the text
filtered_text = preprocess_text(review)
print('Filtered_text:',filtered_text)

# Define the maximum length for padding
max_length = 150  # Adjust as per your model's input shape

# Create and fit a Tokenizer object
token = Tokenizer()
token.fit_on_texts([filtered_text])

# Tokenize the words and pad sequences
tokenize_words = token.texts_to_sequences([filtered_text])
tokenize_words_padded = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')

print(tokenize_words_padded)

# Assuming loaded_model is your pre-trained model
result = loaded_model.predict(tokenize_words_padded)
print(result)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Filtered_text: one reviewers mentioned watching  oz episode youll hooked right exactly happened mebr br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordbr br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awaybr br would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away

In [70]:
print('Text:',filtered_text)
if result >= 0.7:
    print('The Review is positive')
else:
    print('The Review is negative')

Text: one reviewers mentioned watching  oz episode youll hooked right exactly happened mebr br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordbr br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awaybr br would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well man

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer  # Import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model

# Download NLTK stopwords data if not already downloaded
nltk.download('stopwords')

# Define your text preprocessing function
def preprocess_text(text):
    regex = re.compile(r'[^a-zA-Z\s]')
    text = regex.sub('', text)
    words = text.split(' ')
    english_stops = set(stopwords.words('english'))
    filtered = [w.lower() for w in words if w.lower() not in english_stops]
    filtered_text = ' '.join(filtered)
    return filtered_text

# Example text
review = "special effects, just simply good acting and getting simple things right,and MOst importantly--not being LAME--, but i guess this was produced for those Sheeple without taste and not a clue of what is ''A good Movie''don't be scared of rating films low,save your under-appreciated high scores for ''once in a life time movies''. Keep in mind that many use IMDb for trusted reviews and opinions,don't spoil the broth by sugarcoating turds Peace & love"
# Preprocess the text
filtered_text = preprocess_text(review)

# Define the maximum length for padding
max_length = 150  # Adjust as per your model's input shape

# Create and fit a Tokenizer object
token = Tokenizer()
token.fit_on_texts([filtered_text])

# Tokenize the words and pad sequences
tokenize_words = token.texts_to_sequences([filtered_text])
tokenize_words_padded = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')

# Load the pre-trained GRU model
loaded_model = load_model('gru_model.h5')

# Make predictions
result = loaded_model.predict(tokenize_words_padded)
print(result)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1/1 [==============================] - 1s 655ms/step
[[0.60752356]]


In [5]:
print('Text:', filtered_text)
if result >= 0.7:
    print('\033[1mThe Review is positive\033[0m')  # Highlight positive review
else:
    print('\033[1mThe Review is negative\033[0m')  # Highlight negative review


Text: special effects simply good acting getting simple things rightand importantlynot lame guess produced sheeple without taste clue good moviedont scared rating films lowsave underappreciated high scores life time movies keep mind many use imdb trusted reviews opinionsdont spoil broth sugarcoating turds peace  love
The Review is negative


In [23]:

# RNN
def create_rnn_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(SimpleRNN(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# LSTM
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# GRU
def create_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(GRU(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# CNN
def create_cnn_model(filters, kernel_size, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Bidirectional LSTM
def create_bidirectional_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(Bidirectional(LSTM(units)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Bidirectional GRU
def create_bidirectional_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(Bidirectional(GRU(units)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [24]:
units = 64
dropout_rate = 0.5 

In [25]:
# Initialize models
rnn_model = create_rnn_model(units, dropout_rate)
lstm_model = create_lstm_model(units, dropout_rate)
gru_model = create_gru_model(units, dropout_rate)
cnn_model = create_cnn_model(filters=64, kernel_size=3, dropout_rate=0.5)
bidirectional_lstm_model = create_bidirectional_lstm_model(units, dropout_rate)
bidirectional_gru_model = create_bidirectional_gru_model(units, dropout_rate)

In [26]:
# Compile models
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bidirectional_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bidirectional_gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
# Train the models and measure time
start_time = time.time()

In [28]:
# RNN
rnn_history = rnn_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])
# LSTM
lstm_history = lstm_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])
# CNN
cnn_history = cnn_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

# Bidirectional LSTM
bidirectional_lstm_history = bidirectional_lstm_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

# Bidirectional GRU
bidirectional_gru_history = bidirectional_gru_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])


C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
219/219 [==============================] - 112s 511ms/step - loss: 0.5608 - accuracy: 0.7140 - val_loss: 0.4530 - val_accuracy: 0.7997
Epoch 2/10
219/219 [==============================] - 110s 505ms/step - loss: 0.4022 - accuracy: 0.8273 - val_loss: 0.3720 - val_accuracy: 0.8410
Epoch 3/10
219/219 [==============================] - 131s 598ms/step - loss: 0.3652 - accuracy: 0.8474 - val_loss: 0.3794 - val_accuracy: 0.8429
Epoch 4/10
219/219 [==============================] - 221s 1s/step - loss: 0.3393 - accuracy: 0.8598 - val_loss: 0.3846 - val_accuracy: 0.8359
Epoch 5/10
219/219 [==============================] - 221s 1s/step - loss: 0.3154 - accuracy: 0.8715 - val_loss: 0.3812 - val_accuracy: 0.8444
Epoch 1/10
219/219 [==============================] - 408s 2s/step - loss: 0.4673 - accuracy: 0.7759 - val_loss: 0.3459 - val_accuracy: 0.8486
Epoch 2/10
219/219 [==============================] - 447s 2s/step - loss: 0.3320 - accuracy: 0.8601 - val_loss: 0.3568 - val_accurac

In [30]:
end_time = time.time()
training_time = end_time - start_time

In [29]:
# Evaluate models on the test set
rnn_eval = rnn_model.evaluate(X_test_padded, y_test)
lstm_eval = lstm_model.evaluate(X_test_padded, y_test)

cnn_eval = cnn_model.evaluate(X_test_padded, y_test)
bidirectional_lstm_eval = bidirectional_lstm_model.evaluate(X_test_padded, y_test)
bidirectional_gru_eval = bidirectional_gru_model.evaluate(X_test_padded, y_test)



469/469 [==============================] - 326s 695ms/step - loss: 0.3270 - accuracy: 0.8647


In [ ]:
# Extract performance metrics
rnn_test_acc = rnn_eval[1]
lstm_test_acc = lstm_eval[1]

cnn_test_acc = cnn_eval[1]
bidirectional_lstm_test_acc = bidirectional_lstm_eval[1]
bidirectional_gru_test_acc = bidirectional_gru_eval[1]


In [ ]:
# Print performance metrics and training time
print("RNN Test Accuracy:", rnn_test_acc)
print("LSTM Test Accuracy:", lstm_test_acc)

print("CNN Test Accuracy:", cnn_test_acc)
print("Bidirectional LSTM Test Accuracy:", bidirectional_lstm_test_acc)
print("Bidirectional GRU Test Accuracy:", bidirectional_gru_test_acc)
print("Training Time (seconds):", training_time)


In [32]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm

# Save the trained model to an HDF5 file
rnn_model.save('imdbmodel/rnn_model.h5')
cnn_model.save('imdbmodel/cnn_model.h5')
lstm_model.save('imdbmodel/lstm_model.h5')
bidirectional_lstm_model.save('imdbmodel/bidirectional_lstm_model.h5')
bidirectional_gru_model.save('imdbmodel/bidirectional_gru_model.h5')

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer  # Import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model

# Download NLTK stopwords data if not already downloaded
nltk.download('stopwords')

# Define your text preprocessing function
def preprocess_text(text):
    regex = re.compile(r'[^a-zA-Z\s]')
    text = regex.sub('', text)
    words = text.split(' ')
    english_stops = set(stopwords.words('english'))
    filtered = [w.lower() for w in words if w.lower() not in english_stops]
    filtered_text = ' '.join(filtered)
    return filtered_text

# Example text
review = "special effects, just simply good acting and getting simple things right,and MOst importantly--not being LAME--, but i guess this was produced for those Sheeple without taste and not a clue of what is ''A good Movie''don't be scared of rating films low,save your under-appreciated high scores for ''once in a life time movies''. Keep in mind that many use IMDb for trusted reviews and opinions,don't spoil the broth by sugarcoating turds Peace & love"
# Preprocess the text
filtered_text = preprocess_text(review)

# Define the maximum length for padding
max_length = 150  # Adjust as per your model's input shape

# Create and fit a Tokenizer object
token = Tokenizer()
token.fit_on_texts([filtered_text])

# Tokenize the words and pad sequences
tokenize_words = token.texts_to_sequences([filtered_text])
tokenize_words_padded = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')

# Load the pre-trained GRU model
loaded_model = load_model('gru_model.h5')

# Make predictions
result = loaded_model.predict(tokenize_words_padded)
print(result)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1/1 [==============================] - 1s 611ms/step
[[0.60752356]]


In [7]:
print('Text:', filtered_text)
if result >= 0.7:
    print('\033[1mThe Review is positive\033[0m')  # Highlight positive review
else:
    print('\033[1mThe Review is negative\033[0m')  # Highlight negative review


Text: special effects simply good acting getting simple things rightand importantlynot lame guess produced sheeple without taste clue good moviedont scared rating films lowsave underappreciated high scores life time movies keep mind many use imdb trusted reviews opinionsdont spoil broth sugarcoating turds peace  love
The Review is negative


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer  # Import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords data if not already downloaded
nltk.download('stopwords')

# Define your text preprocessing function
def preprocess_text(text):
    regex = re.compile(r'[^a-zA-Z\s]')
    text = regex.sub('', text)
    words = text.split(' ')
    english_stops = set(stopwords.words('english'))
    filtered = [w.lower() for w in words if w.lower() not in english_stops]
    filtered_text = ' '.join(filtered)
    return filtered_text

# Example text
review = "One of the other reviewers has mentioned that after watching just  Oz episode youll be hooked They are right as this is exactly what happened with mebr br The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the wordbr br It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to manyAryans Muslims gangstas Latinos Christians Italians Irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awaybr br I would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare Forget pretty pictures painted for mainstream audiences forget charm forget romanceOZ doesnt mess around The first episode I ever saw struck me as so nasty it was surreal I couldnt say I was ready for it but as I watched more I developed a taste for Oz and got accustomed to the high levels of graphic violence Not just violence but injustice crooked guards wholl be sold out for a nickel inmates wholl kill on order and get away with it well mannered middle class inmates being turned into prison bitches due to their lack of street skills or prison experience Watching Oz you may become comfortable with what is uncomfortable viewingthats if you can get in touch with your darker side"

# Preprocess the text
filtered_text = preprocess_text(review)
print('Filtered_text:',filtered_text)

# Define the maximum length for padding
max_length = 150  # Adjust as per your model's input shape

# Create and fit a Tokenizer object
token = Tokenizer()
token.fit_on_texts([filtered_text])

# Tokenize the words and pad sequences
tokenize_words = token.texts_to_sequences([filtered_text])
tokenize_words_padded = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')

print(tokenize_words_padded)

# Assuming loaded_model is your pre-trained model
result = loaded_model.predict(tokenize_words_padded)
print(result)


Filtered_text: one reviewers mentioned watching  oz episode youll hooked right exactly happened mebr br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordbr br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awaybr br would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
print('Text:', filtered_text)
if result >= 0.7:
    print('\033[1mThe Review is positive\033[0m')  # Highlight positive review
else:
    print('\033[1mThe Review is negative\033[0m')  # Highlight negative review


Text: one reviewers mentioned watching  oz episode youll hooked right exactly happened mebr br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordbr br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awaybr br would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well man